# acell.py

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 10 17:10:22 2019

@author: dhh
"""

import numpy as np
import pandas as pd
#from layer_assist import Unit
#from Unit import call
import tensorflow as tf
dim = 20

2024-10-30 01:42:34.673062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-30 01:42:35.467635: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def load_assist_data():
    sz_adj = pd.read_csv('../DAT/sz_adj.csv', header=None)
    adj = np.mat(sz_adj)
    data = pd.read_csv('../DAT/sz_speed.csv')
    return data, adj

data, adj = load_assist_data()
time_len = data.shape[0]
num_nodes = data.shape[1]

In [5]:
def preprocess_data(data1,  time_len, train_rate, seq_len, pre_len, model_name,scheme):
    train_size = int(time_len * train_rate)
    train_data = data1[0:train_size]
    test_data = data1[train_size:time_len]            
            
    if model_name == 'tgcn':################TGCN###########################
        trainX, trainY, testX, testY = [], [], [], []
        for i in range(len(train_data) - seq_len - pre_len):
            a1 = train_data[i: i + seq_len + pre_len]
            trainX.append(a1[0 : seq_len])
            trainY.append(a1[seq_len : seq_len + pre_len])
        for i in range(len(test_data) - seq_len -pre_len):
            b1 = test_data[i: i + seq_len + pre_len]
            testX.append(b1[0 : seq_len])
            testY.append(b1[seq_len : seq_len + pre_len])         
            
    else:################AST-GCN###########################
        sz_poi = pd.read_csv('../DAT/sz_poi.csv',header = None)
        sz_poi = np.transpose(sz_poi)
        sz_poi_max = np.max(np.max(sz_poi))
        sz_poi_nor = sz_poi/sz_poi_max
        sz_weather = pd.read_csv('../DAT/sz_weather.csv',header = None)
        sz_weather = np.mat(sz_weather)
        sz_weather_max = np.max(np.max(sz_weather))
        sz_weather_nor = sz_weather/sz_weather_max
        sz_weather_nor_train = sz_weather_nor[0:train_size]
        sz_weather_nor_test = sz_weather_nor[train_size:time_len]

        if J == 1:#add poi(dim+1)
            trainX, trainY, testX, testY = [], [], [], []
            for i in range(len(train_data) - seq_len - pre_len):
                a1 = train_data[i: i + seq_len + pre_len]
                a = np.row_stack((a1[0:seq_len],sz_poi_nor[:1]))
                trainX.append(a)
                trainY.append(a1[seq_len : seq_len + pre_len])
            for i in range(len(test_data) - seq_len -pre_len):
                b1 = test_data[i: i + seq_len + pre_len]
                b = np.row_stack((b1[0:seq_len],sz_poi_nor[:1]))
                testX.append(b)
                testY.append(b1[seq_len : seq_len + pre_len])
        if J == 2:#add weather(dim+11)
            trainX, trainY, testX, testY = [], [], [], []
            for i in range(len(train_data) - seq_len - pre_len):
                a1 = train_data[i: i + seq_len + pre_len]
                a2 = sz_weather_nor_train[i: i + seq_len + pre_len]
                a = np.row_stack((a1[0:seq_len],a2[0: seq_len + pre_len]))
                trainX.append(a)
                trainY.append(a1[seq_len : seq_len + pre_len])
            for i in range(len(test_data) - seq_len -pre_len):
                b1 = test_data[i: i + seq_len + pre_len]
                b2 = sz_weather_nor_test[i: i + seq_len + pre_len]
                b = np.row_stack((b1[0:seq_len],b2[0: seq_len + pre_len]))
                testX.append(b)
                testY.append(b1[seq_len : seq_len + pre_len])
        else:#add kg(dim+12)
            trainX, trainY, testX, testY = [], [], [], []
            for i in range(len(train_data) - seq_len - pre_len):
                a1 = train_data[i: i + seq_len + pre_len]
                a2 = sz_weather_nor_train[i: i + seq_len + pre_len]
                a = np.row_stack((a1[0:seq_len],a2[0: seq_len + pre_len],sz_poi_nor[:1]))
                trainX.append(a)
                trainY.append(a1[seq_len : seq_len + pre_len])
            for i in range(len(test_data) - seq_len -pre_len):
                b1 = test_data[i: i + seq_len + pre_len]
                b2 = sz_weather_nor_test[i: i + seq_len + pre_len]
                b = np.row_stack((b1[0:seq_len],b2[0: seq_len + pre_len],sz_poi_nor[:1]))
                testX.append(b)
                testY.append(b1[seq_len : seq_len + pre_len])


    trainX1 = np.array(trainX)
    trainY1 = np.array(trainY)
    testX1 = np.array(testX)

    testY1 = np.array(testY)
    print(trainX1.shape)
    print(trainY1.shape)
    print(testX1.shape)
    print(testY1.shape)
    
    return trainX1, trainY1, testX1, testY1    

In [27]:
time_len = data.shape[0]
num_nodes = data.shape[1]
data1 =np.mat(data,dtype=np.float32)

In [29]:
print(time_len)
print(num_nodes)
print(data.shape)

2976
156
(2976, 156)


In [33]:
data.head()

,90217,90218,90219,90220,90221,90222,90223,90224,90225,90226,...,112746,116097,116098,116099,116100,116157,116219,116220,116221,117189
0,8.471971,18.455410,20.590635,15.345258,9.585218,21.501821,31.611759,0.000000,0.0,22.008941,...,0.000000,27.623514,31.126936,45.680952,18.801449,0.0,22.791096,16.589562,32.590224,0.000000
1,7.807137,15.713816,27.523695,11.087895,9.455280,17.332246,33.976531,12.968823,0.0,17.921757,...,4.193466,14.889713,29.400996,47.128457,18.081014,0.0,22.492316,39.614822,32.588299,0.000000
2,8.809457,8.979647,20.280394,16.523419,8.003314,15.789483,13.747267,12.221143,0.0,21.588447,...,11.076185,0.000000,30.198482,44.345346,17.947455,0.0,22.289410,33.331948,37.471168,0.000000
3,51.590372,23.631243,20.224094,15.116459,6.642644,17.575806,17.657556,15.998745,0.0,24.228905,...,0.000000,34.132564,60.259055,57.122521,35.479929,0.0,40.244356,12.942474,47.140562,39.697219
4,58.770433,20.437740,20.465606,14.820217,11.344404,0.000000,33.878502,5.676248,0.0,26.250064,...,0.000000,15.904281,66.134278,66.451325,37.001600,0.0,37.958725,38.225794,39.550687,0.000000


In [31]:
data1.shape

(2976, 156)

In [36]:
sz_weather = pd.read_csv('../DAT/sz_weather.csv',header = None)
sz_weather = np.mat(sz_weather)
sz_weather.shape

(2976, 156)

In [40]:
data1

matrix([[0.0980221 , 0.21353212, 0.23823702, ..., 0.19194394, 0.37707424,
         0.        ],
        [0.09032986, 0.18181142, 0.31845367, ..., 0.45834997, 0.37705195,
         0.        ],
        [0.10192686, 0.10389599, 0.23464748, ..., 0.3856561 , 0.43354756,
         0.        ],
        ...,
        [0.37947276, 0.141638  , 0.10453826, ..., 0.13605069, 0.1998609 ,
         0.        ],
        [0.39722532, 0.18600048, 0.121989  , ..., 0.16306393, 0.16957766,
         0.        ],
        [0.38101357, 0.14001252, 0.10420388, ..., 0.19054307, 0.1845446 ,
         0.        ]], dtype=float32)

In [41]:
sz_weather

matrix([[2, 2, 2, ..., 2, 2, 2],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2]])

In [43]:
sz_weather.shape

(2976, 156)

In [38]:
a = np.row_stack((data1, sz_weather))

In [39]:
a.shape

(5952, 156)

In [42]:
a

matrix([[0.0980221 , 0.21353212, 0.23823702, ..., 0.19194394, 0.37707424,
         0.        ],
        [0.09032986, 0.18181142, 0.31845367, ..., 0.45834997, 0.37705195,
         0.        ],
        [0.10192686, 0.10389599, 0.23464748, ..., 0.38565609, 0.43354756,
         0.        ],
        ...,
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ],
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ],
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ]])

In [34]:
#### normalization
max_value = np.max(data1)
data1  = data1/max_value

data1.columns = data.columns
if model_name == 'ast-gcn':
    if scheme == 1:
        name = 'add poi dim'
    elif scheme == 2:
        name = 'add weather dim'
    else:
        name = 'add poi + weather dim'

else:
    name = 'tgcn'

print('model:', model_name)
print('scheme:', name)
print('noise_name:', noise_name)
print('noise_param:', PG)

trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len, model_name, scheme)

totalbatch = int(trainX.shape[0]/batch_size)
training_data_count = len(trainX)

NameError: name 'model_name' is not defined

In [6]:
class Unit():
    def __init__(self, dim, num_nodes, reuse = None):
#        super(Unit, self).__init__(_reuse=reuse)
        self.dim = dim
        self.num_nodes = num_nodes
    def call(self, inputs, time_len):
        x, e = inputs  
        unit_matrix1 = tf.matmul(x, e)
        unit_matrix = tf.convert_to_tensor(unit_matrix1)
        self.weight_unit ,self.bias_unit = self._emb(dim, time_len)
        
        x1 = tf.matmul(tf.cast(unit_matrix,tf.float32),self.weight_unit)
        x_output = tf.add(x1, self.bias_unit)
#        x_output = pd.DataFrame(x_output)
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes, time_len])
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes * time_len])
        return x_output
    
    def _emb(self, dim, time_len):
        
        with tf.variable_scope('a',reuse = tf.AUTO_REUSE):
            weight_unit = tf.get_variable(name = 'weight_unit', shape = (self.dim, self.num_nodes), dtype = tf.float32)
            bias_unit = tf.get_variable(name = 'bias_unit', shape =(time_len,1), initializer = tf.constant_initializer(dtype=tf.float32))
        w = weight_unit
        b = bias_unit
        return w, b
#
#
#

In [7]:
class Unit1():
    def __init__(self, dim, num_nodes, reuse = None):
#        super(Unit, self).__init__(_reuse=reuse)
        self.dim = dim
        self.num_nodes = num_nodes
    def call(self, inputs, time_len):
        x, e = inputs  
        unit_matrix1 = tf.matmul(x, e)
        unit_matrix=tf.convert_to_tensor(unit_matrix1)
        self.weight_unit1 ,self.bias_unit1 = self._emb(dim, time_len)
        
        x1=tf.matmul(tf.cast(unit_matrix,tf.float32),self.weight_unit1)
        x_output= tf.add(x1, self.bias_unit1)
#        x_output = pd.DataFrame(x_output)
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes, time_len])
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes * time_len])
        return x_output
    
    def _emb(self, dim, time_len):
        
        with tf.variable_scope('a',reuse = tf.AUTO_REUSE):
            weight_unit1 = tf.get_variable(name = 'weight_unit1', shape = (self.dim, self.num_nodes), dtype = tf.float32)
            bias_unit1 = tf.get_variable(name = 'bias_unit1', shape =(time_len,1), initializer = tf.constant_initializer(dtype=tf.float32))
        w1 = weight_unit1
        bb = bias_unit1
        return w1, bb

In [8]:
class Unit2():
    def __init__(self, dim, num_nodes, time_len, reuse = None):
#        super(Unit, self).__init__(_reuse=reuse)
        self.dim = dim
        self.num_nodes = num_nodes
        self.time_len = time_len
    def call(self, inputs, time_len):
        x, e = inputs
        x = np.transpose(x)
        x = x.astype(np.float64)
        unit_matrix1 = tf.matmul(x, e)
        unit_matrix = tf.convert_to_tensor(unit_matrix1)
        self.weight_unit ,self.bias_unit = self._emb(dim, time_len)
        
        x1 = tf.matmul(tf.cast(unit_matrix,tf.float32),self.weight_unit)
        self.x_output = tf.add(x1, self.bias_unit)
#        print(x_output)
        self.x_output = tf.transpose(self.x_output)
#        x = x.astype(np.float64)
#        x_output = pd.DataFrame(x_output)
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes, time_len])
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes * time_len])
        return self.x_output
    
    def _emb(self, dim, time_len):
        
        with tf.variable_scope('a',reuse = tf.AUTO_REUSE):
            self.weight_unit = tf.get_variable(name = 'weight_unit', shape = (self.dim, self.time_len), dtype = tf.float32)
            self.bias_unit = tf.get_variable(name = 'bias_unit', shape =(num_nodes,1), initializer = tf.constant_initializer(dtype=tf.float32))
        self.w = self.weight_unit
        self.b = self.bias_unit
        return self.w, self.b

In [9]:
class Unit3():
    def __init__(self, dim, num_nodes, time_len, reuse = None):
#        super(Unit, self).__init__(_reuse=reuse)
        self.dim = dim
        self.num_nodes = num_nodes
        self.time_len = time_len
    def call(self, inputs, time_len):
        x, e = inputs
        x = np.transpose(x)
        x = x.astype(np.float64)
        unit_matrix1 = tf.matmul(x, e)
        unit_matrix = tf.convert_to_tensor(unit_matrix1)
        self.weight_unit ,self.bias_unit = self._emb(dim, time_len)
        
        x1 = tf.matmul(tf.cast(unit_matrix,tf.float32),self.weight_unit)
        x_output = tf.add(x1, self.bias_unit)
        x_output = tf.transpose(x_output)
#        x_output = pd.DataFrame(x_output)
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes, time_len])
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes * time_len])
        return x_output
    
    def _emb(self, dim, time_len):
        
        with tf.variable_scope('a',reuse = tf.AUTO_REUSE):
            weight_unit = tf.get_variable(name = 'weight_unit', shape = (self.dim, self.time_len), dtype = tf.float32)
            bias_unit = tf.get_variable(name = 'bias_unit', shape =(num_nodes,1), initializer = tf.constant_initializer(dtype=tf.float32))
        w = weight_unit
        b = bias_unit
        return w, b

In [10]:
class Unit4():
    def __init__(self, dim, num_nodes, reuse = None):
#        super(Unit, self).__init__(_reuse=reuse)
        self.dim = dim
        self.num_nodes = num_nodes
    def call(self, inputs, time_len):
        x, e = inputs  
        unit_matrix1 = tf.matmul(x, e)
        unit_matrix = tf.convert_to_tensor(unit_matrix1)
        self.weight_unit ,self.bias_unit = self._emb(dim, time_len)
        
        x1 = tf.matmul(tf.cast(unit_matrix,tf.float32),self.weight_unit)
        x_output = tf.add(x1, self.bias_unit)
#        x_output = pd.DataFrame(x_output)
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes, time_len])
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes * time_len])
        return x_output
    
    def _emb(self, dim, time_len):
        
        with tf.variable_scope('a',reuse = tf.AUTO_REUSE):
            weight_unit = tf.get_variable(name = 'weight_unit', shape = (self.dim, self.num_nodes), dtype = tf.float32)
            bias_unit = tf.get_variable(name = 'bias_unit', shape =(time_len,1), initializer = tf.constant_initializer(dtype=tf.float32))
        w = weight_unit
        b = bias_unit
        return w, b

In [11]:
class Unit5():
    def __init__(self, dim, num_nodes, reuse = None):
#        super(Unit, self).__init__(_reuse=reuse)
        self.dim = dim
        self.num_nodes = num_nodes
    def call(self, inputs, time_len):
        x, e = inputs  
        unit_matrix1 = tf.matmul(x, e)
        unit_matrix=tf.convert_to_tensor(unit_matrix1)
        self.weight_unit1 ,self.bias_unit1 = self._emb(dim, time_len)
        
        x1=tf.matmul(tf.cast(unit_matrix,tf.float32),self.weight_unit1)
        self.x_output= tf.add(x1, self.bias_unit1)
#        x_output = pd.DataFrame(x_output)
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes, time_len])
#        x_output = tf.reshape(x_output, shape=[-1, self.num_nodes * time_len])
        return self.x_output
    
    def _emb(self, dim, time_len):
        
        with tf.variable_scope('a',reuse = tf.AUTO_REUSE):
            self.weight_unit1 = tf.get_variable(name = 'weight_unit1', shape = (self.dim, self.num_nodes), dtype = tf.float32)
            self.bias_unit1 = tf.get_variable(name = 'bias_unit1', shape =(time_len,1), initializer = tf.constant_initializer(dtype=tf.float32))
        self.w1 = self.weight_unit1
        self.bb = self.bias_unit1
        return self.w1, self.bb

# utils.py

In [18]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import scipy.sparse as sp
#from scipy.sparse import linalg
import numpy as np


def normalized_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    normalized_adj = normalized_adj.astype(np.float32)
    return normalized_adj
    
def sparse_to_tuple(mx):
    mx = mx.tocoo()
    coords = np.vstack((mx.row, mx.col)).transpose()
    L = tf.SparseTensor(coords, mx.data, mx.shape)
    return tf.sparse_reorder(L) 
    
def calculate_laplacian(adj, lambda_max=1):  
    adj = normalized_adj(adj + sp.eye(adj.shape[0]))
    adj = sp.csr_matrix(adj)
    adj = adj.astype(np.float32)
    return sparse_to_tuple(adj)
    
def weight_variable_glorot(input_dim, output_dim, name=""):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.random_uniform([input_dim, output_dim], minval=-init_range,
                            maxval=init_range, dtype=tf.float32)

    return tf.Variable(initial,name=name)  

# tgcn.py

In [15]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/49/df/c6e5ef56bed7c621763867f38aabd860aa55d7da27bccfc72f4816d9969f/tensorflow-2.18.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tensorboard<2.19,>=2.18 from https://files.pythonhosted.org/packages/b1/de/021c1d407befb505791764ad2cbd56ceaaa53a746baed01d2e2143f05f18/tensorboard-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for keras>=3.5.0 from https://files.pythonhosted.org/packages/c2/88/eef50051a772dcb4433d1f3e4c1d6576ba450fe83e89d028d7e8b85a2122/keras-3.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for numpy<2.1.0,>=1.26.0 from https://files.pythonhosted.org/packages/b9/14

In [25]:
# -*- coding: utf-8 -*-

#import numpy as np
import tensorflow as tf
from tensorflow import keras
# from tensorflow.contrib.rnn import RNNCell
from tensorflow.python.keras.layers import GRUCell, RNN

# from utils import calculate_laplacian
import numpy as np
import pandas as pd

In [ ]:
class tgcnCell(tf.keras.layers.Layer):
    """Temporal Graph Convolutional Network """

    def call(self, inputs, **kwargs):
        pass

    def __init__(self, num_units, adj, num_nodes, input_size=None,
                 act=tf.nn.tanh, reuse=None):

        super(tgcnCell, self).__init__(_reuse=reuse)
        self._act = act
        self._nodes = num_nodes
        self._units = num_units
        self._adj = []
        self._adj.append(calculate_laplacian(adj))


    @property
    def state_size(self):
        return self._nodes * self._units

    @property
    def output_size(self):
        return self._units

    def __call__(self, inputs, state, scope=None):

        with tf.variable_scope(scope or "tgcn"):
            with tf.variable_scope("gates"):  
                value = tf.nn.sigmoid(
                    self._gc(inputs, state, 2 * self._units, bias=1.0, scope=scope))
                r, u = tf.split(value=value, num_or_size_splits=2, axis=1)
            with tf.variable_scope("candidate"):
                r_state = r * state
                c = self._act(self._gc(inputs, r_state, self._units, scope=scope))
            new_h = u * state + (1 - u) * c
        return new_h, new_h


    def _gc(self, inputs, state, output_size, bias=0.0, scope=None):
        ## inputs:(-1,num_nodes)
        inputs = tf.expand_dims(inputs, 2)
#        print('inputs_shape:',inputs.shape)
        ## state:(batch,num_node,gru_units)
        state = tf.reshape(state, (-1, self._nodes, self._units))
#        print('state_shape:',state.shape)
        ## concat
        x_s = tf.concat([inputs, state], axis=2)
#        print('x_s_shape:',x_s.shape)
        
        
#        kgembedding = np.array(pd.read_csv(r'/DHH/sz_gcn/sz_data/sz_poi_transR_embedding20.csv',header=None))
#        kgeMatrix = np.repeat(kgembedding[np.newaxis, :, :], self._units, axis=0)
#        kgeMatrix = tf.reshape(tf.constant(kgeMatrix, dtype=tf.float32), (self._units, -1))
#        kgMatrix = tf.reshape(kgeMatrix,(-1,self._nodes, 20))
#        
#        ## inputs:(-1,num_nodes)
#        inputs = tf.expand_dims(inputs, 2)
#        ## state:(batch,num_node,gru_units)
#        state = tf.reshape(state, (-1, self._nodes, self._units))
#        ## concat
#        print('kgMatrix_shape:',kgMatrix.shape)
#        print('inputs_shape:',inputs.shape)
#        print('state_shape:',state.shape)
#        kg_x = tf.concat([inputs, kgMatrix],axis = 2)
#        print('kg_x_shape:',kg_x.shape)
#        x_s = tf.concat([kg_x, state], axis=2)
        input_size = x_s.get_shape()[2].value
        ## (num_node,input_size,-1)
        x0 = tf.transpose(x_s, perm=[1, 2, 0])  
        x0 = tf.reshape(x0, shape=[self._nodes, -1])
        
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope):
            for m in self._adj:
                x1 = tf.sparse_tensor_dense_matmul(m, x0)
#                print(x1)
            x = tf.reshape(x1, shape=[self._nodes, input_size,-1])
            x = tf.transpose(x,perm=[2,0,1])
            x = tf.reshape(x, shape=[-1, input_size])
            weights = tf.get_variable(
                'weights', [input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
            x = tf.matmul(x, weights)  # (batch_size * self._nodes, output_size)
            biases = tf.get_variable(
                "biases", [output_size], initializer=tf.constant_initializer(bias, dtype=tf.float32))
            x = tf.nn.bias_add(x, biases)
            x = tf.reshape(x, shape=[-1, self._nodes, output_size])
            x = tf.reshape(x, shape=[-1, self._nodes * output_size])
        return x

In [ ]:
# -*- coding: utf-8 -*-

import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os
import numpy.linalg as la
from acell import preprocess_data,load_assist_data
from tgcn import tgcnCell

from visualization import plot_result,plot_error
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time

time_start = time.time()
###### Settings ######
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_integer('training_epoch', 3000, 'Number of epochs to train.')
flags.DEFINE_integer('gru_units', 128, 'hidden units of gru.')
flags.DEFINE_integer('seq_len',10 , 'time length of inputs.')
flags.DEFINE_integer('pre_len', 3, 'time length of prediction.')
flags.DEFINE_float('train_rate', 0.8, 'rate of training set.')
flags.DEFINE_integer('batch_size', 64, 'batch size.')
flags.DEFINE_string('dataset', 'sz', 'dataset')
flags.DEFINE_string('model_name', 'tgcn', 'ast-gcn')
flags.DEFINE_string('scheme', 1, 'scheme')
flags.DEFINE_string('noise_name', 'None', 'None or Gauss or Possion')
flags.DEFINE_string('noise_param', 0, 'Parameter for noise')

model_name = FLAGS.model_name
noise_name = FLAGS.noise_name
data_name = FLAGS.dataset
train_rate =  FLAGS.train_rate
seq_len = FLAGS.seq_len
output_dim = pre_len = FLAGS.pre_len
batch_size = FLAGS.batch_size
lr = FLAGS.learning_rate
training_epoch = FLAGS.training_epoch
gru_units = FLAGS.gru_units
dim = FLAGS.dim
scheme = FLAGS.scheme
PG = FLAGS.noise_param

###### load data ######
if data_name == 'sz':
    data, adj = load_assist_data('sz')
#if data_name == 'sh':
#    data, adj = load_sh_data('sh')
### Perturbation Analysis
def MaxMinNormalization(x,Max,Min):
    x = (x-Min)/(Max-Min)
    return x

if noise_name == 'Gauss':
    Gauss = np.random.normal(0,PG,size=data.shape)
    noise_Gauss = MaxMinNormalization(Gauss,np.max(Gauss),np.min(Gauss))
    data = data + noise_Gauss
elif noise_name == 'Possion':
    Possion = np.random.poisson(PG,size=data.shape)
    noise_Possion = MaxMinNormalization(Possion,np.max(Possion),np.min(Possion))
    data = data + noise_Possion
else:
    data = data

time_len = data.shape[0]
num_nodes = data.shape[1]
data1 =np.mat(data,dtype=np.float32)

#### normalization
max_value = np.max(data1)
data1  = data1/max_value

data1.columns = data.columns
if model_name == 'ast-gcn':
    if scheme == 1:
        name = 'add poi dim'
    elif scheme == 2:
        name = 'add weather dim'
    else:
        name = 'add poi + weather dim'

else:
    name = 'tgcn'

print('model:', model_name)
print('scheme:', name)
print('noise_name:', noise_name)
print('noise_param:', PG)

trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len, model_name, scheme)

totalbatch = int(trainX.shape[0]/batch_size)
training_data_count = len(trainX)

def TGCN(_X, _weights, _biases):
    ###
    cell_1 = tgcnCell(gru_units, adj, num_nodes=num_nodes)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    outputs, states = tf.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,gru_units])
        o = tf.reshape(o,shape=[-1,gru_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, _weights['out']) + _biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])
    return output, m, states
    
    
###### placeholders ######
if model_name == 'ast-gcn':
    if scheme == 1:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len+1, num_nodes])
    elif scheme == 2:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len*2+pre_len, num_nodes])
    else:
        inputs = tf.placeholder(tf.float32, shape=[None, seq_len*2+pre_len+1, num_nodes])

else:
    inputs = tf.placeholder(tf.float32, shape=[None, seq_len, num_nodes])

labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])

# Graph weights
weights = {
    'out': tf.Variable(tf.random_normal([gru_units, pre_len], mean=1.0), name='weight_o')}
biases = {
    'out': tf.Variable(tf.random_normal([pre_len]),name='bias_o')}

pred,ttts,ttto = TGCN(inputs, weights, biases)

y_pred = pred
      

###### optimizer ######
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])
##loss
print('y_pred_shape:', y_pred.shape)
print('label_shape:', label.shape)
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)
##rmse
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

###### Initialize session ######
variables = tf.global_variables()
saver = tf.train.Saver(tf.global_variables())  
#sess = tf.Session()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.global_variables_initializer())

#out = 'out/%s'%(model_name)
out = 'out/%s_%s'%(model_name,noise_name)
path1 = '%s_%s_%s_lr%r_batch%r_unit%r_seq%r_pre%r_epoch%r_scheme%r_PG%r'%(model_name,name,data_name,lr,batch_size,gru_units,seq_len,pre_len,training_epoch,scheme,PG)
path = os.path.join(out,path1)
if not os.path.exists(path):
    os.makedirs(path)
    
###### evaluation ######
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, 1-F_norm, r2, var
 
   
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_acc,test_r2,test_var,test_pred = [],[],[],[],[],[],[]
  
for epoch in range(training_epoch):
    for m in range(totalbatch):
        mini_batch = trainX[m * batch_size : (m+1) * batch_size]
        mini_label = trainY[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        batch_rmse.append(rmse1 * max_value)

     # Test completely at every epoch
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:testX, labels:testY})

    testoutput = np.abs(test_output)
    test_label = np.reshape(testY,[-1,num_nodes])
    rmse, mae, acc, r2_score, var_score = evaluation(test_label, testoutput)
    test_label1 = test_label * max_value
    test_output1 = testoutput * max_value
    test_loss.append(loss2)
    test_rmse.append(rmse * max_value)
    test_mae.append(mae * max_value)
    test_acc.append(acc)
    test_r2.append(r2_score)
    test_var.append(var_score)
    test_pred.append(test_output1)
    
    print('Iter:{}'.format(epoch),
          'train_rmse:{:.4}'.format(batch_rmse[-1]),
          'test_loss:{:.4}'.format(loss2),
          'test_rmse:{:.4}'.format(rmse),
          'test_acc:{:.4}'.format(acc))
    
    if (epoch % 500 == 0):        
        saver.save(sess, path+'/model_100/ASTGCN_pre_%r'%epoch, global_step = epoch)
        
time_end = time.time()
print(time_end-time_start,'s')

############## visualization ###############
#x = [i for i in range(training_epoch)]
b = int(len(batch_rmse)/totalbatch)
batch_rmse1 = [i for i in batch_rmse]
train_rmse = [(sum(batch_rmse1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
batch_loss1 = [i for i in batch_loss]
train_loss = [(sum(batch_loss1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
#test_rmse = [float(i) for i in test_rmse]

index = test_rmse.index(np.min(test_rmse))
test_result = test_pred[index]
var = pd.DataFrame(test_result)
var.to_csv(path+'/test_result.csv',index = False,header = False)
plot_result(test_result,test_label1,path)
plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path)
evalution = []
evalution.append(np.min(test_rmse))
evalution.append(test_mae[index])
evalution.append(test_acc[index])
evalution.append(test_r2[index])
evalution.append(test_var[index])
evalution = pd.DataFrame(evalution)
evalution.to_csv(path+'/evalution.csv',index=False,header=None)
print('model_name:', model_name)
print('scheme:', scheme)
print('name:', name)
print('noise_name:', noise_name)
print('PG:', PG)
print('min_rmse:%r'%(np.min(test_rmse)),
      'min_mae:%r'%(test_mae[index]),
      'max_acc:%r'%(test_acc[index]),
      'r2:%r'%(test_r2[index]),
      'var:%r'%test_var[index])